In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%run "/content/drive/MyDrive/Research/Masters Thesis/V1/0. Parent Code.ipynb"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2401


# 1. Loading Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from statistics import mean

import scipy
from scipy.sparse import csr_matrix

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#2. Select the required texts from LinkedIn

In [ ]:
#Create labeled and unlabeled text list 


# sex, name, degree_connection, current_location, current_industry, number_connection, 
# edu1, edu2, edu3, edu4, edu5, edu6, 
# number_skills, skills_list, skills_endorsement_pair, Intersts, summary,
# employment

# Education is iteself and object with the following attributes
# SName, SLevel, SDates, SField, SDegree

#Here employment is a list of jobs where each job contains information below 
# [title, company_name, job_dates, job_location, job_summary, job_skills]


labeled_list = []
labeled_list_label = []
unlabeled_list = []
object_number =0
info_list_labeled = []
info_list_unlabeled = []
for ind in object_list:

    for job in ind.employment:
        if job[4]!=None and job[5]!=None and len(job[5]) >= 5:
            split_description = job[4].split("\n skill : ")
            if len(split_description) == 2 :
                info_list_labeled.append([ object_number ]+ job + [ind.skills_endorsement_pair])
                labeled_list.append(job[4])
                labeled_list_label.append(job[5])
            else:
                print(split_description)
                a = "Error Check" + 1
            
        elif  job[4]!=None: 
            info_list_unlabeled.append([ object_number ]+ job)
            unlabeled_list.append(job[4])
    
    object_number = object_number  +1

    # print("______________________")

print(len(labeled_list))
print(len(info_list_labeled))
print(len(unlabeled_list))
print(len(info_list_unlabeled))

1171
1171
14182
14182


In [ ]:
info_list_labeled[567]

# 3. Creating Y

## 3.1 Setting things up 

### 3.1.1 ONET Labels

In [ ]:
ONET_df = pd.read_csv(main_link  + "Input/final_ONET.csv")
ONET_column_type = ["Abilities", "Work Context", "Skills", "Knowledge", "Work Activities","Description", 	"Rating"]
selected_ONET_column = [x for x in ONET_df.columns if x.split("_")[0] in ONET_column_type ]

ONET_df = ONET_df[selected_ONET_column]

In [ ]:
#Applying weight to the text descriptions 

def weigh_description(x):
  #Applying weight to different stated task 
  #The descriptions are repeated 10 times 
  description = (x["Description"] + " \n ")*10
  #The task statment are repeated by the weight which are rounded to the nearest 10
  task_description = ""
  for i in x["Rating_task_values"][2:-2].split("], ["):
    i_split = i.split("', ")
    if len(i_split) != 2 : i_split = i.split('", ')
    task_i = i_split[0][1:]
    rating_i = round(float(i_split[1])/10 ) 
    task_description = task_description + " \n " + (task_i  + " \n ")*rating_i 
  #Output is a combination of the two
  output = description  + " \n " + task_description
  return output 

ONET_df['ONET_text'] = ONET_df.apply(lambda x : weigh_description(x), axis = 1)



### 3.1.2 LinkedIn labeled discription labels

In [ ]:
#Create dictionary of LinkedIn skills to ONET skills from brute force sheet 
brute_force = pd.read_excel(main_link  + "Input/Brute force Final.xlsx" )

brute_force_values = brute_force[["Value _" + str(i) for i in range(1,18)]].values.tolist()
brute_force_values = [[y for y in x if str(y) != "nan"] for x in brute_force_values]
brute_force_keys = [x.strip() for x in brute_force.LinkedIn_skills.values.tolist()]



In [ ]:
#Create a dictionary of LinkedIn Skills to ONET skills 
#Brute force only has three skill groups (Skills, abilities, knowledge), adding information on (Work Activities, Work Context) using their crosswalk


def Dict_WAC_crosswalk(df, c1, c2):
  WAC = c2.split(" ")[0] + " " +c2.split(" ")[1]     #WAC is either Work Context or Work Activities 
  skill_or_ability = c1.split(" ")[0]                 #This is either skills or abilities 
  Dict_output = {}

  for i in df[[c1, c2]].values.tolist():             
    item_to_add_value  = WAC+ "_"+i[1]                    #Example : Work Activities_Getting Information
    item_to_add_key = skill_or_ability + "_"+ i[0]             #Example: Skills_Reading Comprehension

    if item_to_add_key in Dict_output.keys():     #If an entry for this skills or activity has already been created 
      current_value = Dict_output[item_to_add_key]
      current_value.append(item_to_add_value )
      Dict_output[item_to_add_key] = current_value
    else:
      Dict_output[item_to_add_key] = [ item_to_add_value ]

  output = Dict_output 
  return output 

Skills_to_WA = Dict_WAC_crosswalk(  pd.read_excel(main_link  + "Input/O*NET Input/Data_xlsx/Skills to Work Activities.xlsx")      , "Skills Element Name", "Work Activities Element Name") 
Skills_to_WC = Dict_WAC_crosswalk(pd.read_excel(main_link  + "Input/O*NET Input/Data_xlsx/Skills to Work Context.xlsx")           , "Skills Element Name", "Work Context Element Name" )
Abilities_to_WA = Dict_WAC_crosswalk(pd.read_excel(main_link  + "Input/O*NET Input/Data_xlsx/Abilities to Work Activities.xlsx")  , "Abilities Element Name","Work Activities Element Name")
Abilities_to_WC = Dict_WAC_crosswalk(pd.read_excel(main_link  + "Input/O*NET Input/Data_xlsx/Abilities to Work Context.xlsx")     ,"Abilities Element Name","Work Context Element Name" )

#Zipping skills, abilities and knowledge into a dictionary
Dict_labeled_skills_ONET = dict(zip(brute_force_keys, brute_force_values))

#Updating the values to included Work Actitivites and Work Context
WAC_dict_list = [Skills_to_WA, Skills_to_WC, Abilities_to_WA, Abilities_to_WC]

print(list(Dict_labeled_skills_ONET.keys()))


for key in list(Dict_labeled_skills_ONET.keys())[0:1]:  #key here is Linkedin Skills 
  current_value = Dict_labeled_skills_ONET[key]    #current_value is a list of linked skills, attitudes and knowledge 

  for item in Dict_labeled_skills_ONET[key]:       #here item is the individual linked skills, attitudes and knowledge
    for k in WAC_dict_list:                        #Now check if item is a key for any of the WA WC dictionaries
      if item in k.keys():
        current_value.extend(k[item])              #if item is in a dictionary add values of the WA WC dictionaries to current_value 
  Dict_labeled_skills_ONET[key] = list(set(current_value))    #Finally update with this new value


#Dict_labeled_skills_ONET["Government"]

['Business Development', 'Employee Benefits', 'Government', 'Organizational Development', 'Portfolio Management', 'Public Policy', 'Army', 'Call Centers', 'Cold Calling', 'Navy', 'Public Speaking', 'Sports', 'Physical Security', 'Security', 'Typing', 'Dreamweaver', 'Lean Six Sigma', 'ITIL', 'OS X', 'Copywriting', 'Scrum', 'Six Sigma', 'SDLC', 'Administration', 'Administrative Assistance', 'Administrative Assistants', 'Agile & Waterfall Methodologies', 'Agile Methodologies', 'Analysis', 'Analytical Skills', 'Benefits Administration', 'Blogging', 'Calendaring', 'Change Management', 'Cisco Technologies', 'Clinical Research', 'Collaborative Problem Solving', 'Community Outreach', 'Content Development', 'Content Management', 'Content Strategy', 'Copy Editing', 'Corporate Communications', 'Counterterrorism', 'Creative Direction', 'Creative Problem Solving', 'Creative Strategy', 'Creative Writing', 'Crisis Management', 'Critical Thinking', 'Data Analytics', 'Data Center', 'Data Science', 'Dat

In [ ]:
ONET_column_type = ["Abilities", "Work Context", "Skills", "Knowledge", "Work Activities"]
selected_ONET_column = [x for x in ONET_df.columns if x.split("_")[0] in ONET_column_type ]


In [ ]:
skill_measures = []

for job in info_list_labeled:
  # Initialize Dictionary
  Dict_skills_4dis_job =  dict(zip(selected_ONET_column, [0 for x in range(0,len(selected_ONET_column))])) 

  #Get endorsment information
  endorsments = [x[1] for x in job[-1]]
  highest_endorsment = max(endorsments )
  mean_endorsment = sum(endorsments ) / len(endorsments )
  skill_endorsment = dict(job[-1])

  for skill in job[-2]:
    if skill in Dict_labeled_skills_ONET.keys(): #If this is a common enough skill
      ONET_skills = Dict_labeled_skills_ONET[skill]  #Get the linked ONET skills 
      if skill in skill_endorsment.keys() and highest_endorsment >0: weight = 0.5 + (skill_endorsment[skill]+1) / (2 * highest_endorsment)
      else: weight = 0.74 
      for i in ONET_skills: 
        if Dict_skills_4dis_job[i] < weight : Dict_skills_4dis_job[i] = weight 
  skill_measures.append([job[5]]+[Dict_skills_4dis_job[x] for x in selected_ONET_column])


In [ ]:
LinkedIn_labeled_skills_df = pd.DataFrame(skill_measures, columns = ['ONET_text']+selected_ONET_column )


### 3.1.3 Final merge of total labeled data

In [ ]:
total_labeled_data = pd.concat([ONET_df , LinkedIn_labeled_skills_df])


## 3.2 Finding the best classifier of Y

Use ONET to train and labeled LinkedIn to test

In [ ]:
LinkedIn_labeled_skills_df

In [ ]:

#Input = ONET_df, LinkedIn_labeled_skills_df
#Output = Dict of accuracy of each classifier

from collections import Counter


def test_train_df(df_train, df_test):

  test, train = df_test ,  df_train
  X_train = train.ONET_text
  X_test = test.ONET_text

  return test, train, X_test, X_train



def change_to_percentile(x):
    if x<0.25: output = "0.25" 
    elif x<0.50: output = "0.50"
    elif x<0.75: output = "0.75"
    else: output = "1"
    return output

def specify_pipelines():
  #Multinomial Naives Bayes's Pipleline
  NB_pipeline = Pipeline([
                  ('tfidf', TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 2))),
                  ('clf', OneVsRestClassifier(MultinomialNB(
                      fit_prior=True, class_prior=None))),
              ])
  #Linear Support Vector Classification's Pipeline
  SVC_pipeline = Pipeline([
                  ('tfidf', TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 2))),
                  ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
              ])
  #Multinomial Logistic's Pipeline
  LogReg_pipeline = Pipeline([
                  ('tfidf', TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 2))),
                  ('clf', OneVsRestClassifier(LogisticRegression(solver='sag', max_iter = 500), n_jobs=1)),
              ])
  return NB_pipeline, SVC_pipeline, LogReg_pipeline






def find_best_classifier(ONET_df, LinkedIn_labeled_skills_df, ONET_skills_column):

  #set pipelines 
  NB_pipeline, SVC_pipeline,  LogReg_pipeline = specify_pipelines()

  #Find test accuracy 
  test_accuracy_NB ,test_accuracy_SVC ,test_accuracy_Logistic = [], [], []
  a = 0
  
  Y_skills_list = []
  for skills in ONET_skills_column:
      
      #create test train for this skill
      count = Counter(LinkedIn_labeled_skills_df[skills].values.tolist()).most_common(4)
      if len(count)>1 and (count[0][1] < 700):
        print(skills, count)
        LinkedIn_train, LinkedIn_test = train_test_split(LinkedIn_labeled_skills_df, random_state=42, test_size=0.33, shuffle=True)
        test, train, X_test, X_train = test_train_df(     pd.concat([ONET_df , LinkedIn_train])   ,  LinkedIn_test )
        
        print(a)
        a =a +1
        print('... Processing {}'.format(skills))

        # train the model using X_dtm & y
        y = np.array([change_to_percentile(xi) for xi in train[skills]])
        
        NB_pipeline.fit(X_train,  y  )
        SVC_pipeline.fit(X_train,  y  )
        LogReg_pipeline.fit(X_train, y)

      
        # compute the testing accuracy
        prediction_NB = NB_pipeline.predict(X_test)
        prediction_SVC = SVC_pipeline.predict(X_test)
        prediction_LogReg = LogReg_pipeline.predict(X_test)

        y = np.array([change_to_percentile(xi) for xi in test[skills]])

        NB_accuracy, SVC_accuracy , Logstic_accuracy = accuracy_score(y, prediction_NB), accuracy_score(y, prediction_SVC), accuracy_score(y, prediction_LogReg)
        print('Test accuracy for NB is {}'.format(NB_accuracy))
        print('Test accuracy for SVC is {}'.format(SVC_accuracy ))
        print('Test accuracy for Logistic is {}'.format(Logstic_accuracy))

        test_accuracy_NB.append(NB_accuracy)
        test_accuracy_SVC.append(SVC_accuracy)
        test_accuracy_Logistic.append(Logstic_accuracy)
        Y_skills_list.append(skills)

  print("                 ")

  Dict_NB_accuracy = dict(zip(Y_skills_list, test_accuracy_NB))
  Dict_SVC_accuracy = dict(zip(Y_skills_list, test_accuracy_SVC))
  Dict_Logistic_accuracy = dict(zip(Y_skills_list, test_accuracy_Logistic))
  
  return Dict_NB_accuracy, Dict_SVC_accuracy, Dict_Logistic_accuracy






#Skills to consider
ONET_column_type_skills = ["Abilities", "Work Context", "Skills", "Knowledge", "Work Activities"]
ONET_skills_column = [x for x in ONET_df.columns if x.split("_")[0] in ONET_column_type_skills]


Dict_NB_accuracy , Dict_SVC_accuracy, Dict_Logistic_accuracy = find_best_classifier(ONET_df, LinkedIn_labeled_skills_df, ONET_skills_column)

print("NB -> " + str(mean(Dict_NB_accuracy.values())))
print("SVC -> " + str(mean(Dict_SVC_accuracy.values())))
print("Logistic -> " + str(mean(Dict_Logistic_accuracy.values())))

Abilities_Inductive Reasoning [(0.0, 485), (0.74, 75), (1.0, 47), (0.5714285714285714, 42)]
0
... Processing Abilities_Inductive Reasoning
Test accuracy for NB is 0.4728682170542636
Test accuracy for SVC is 0.6511627906976745
Test accuracy for Logistic is 0.5658914728682171
Abilities_Mathematical Reasoning [(0.0, 689), (0.74, 50), (1.0, 33), (0.5714285714285714, 27)]
1
... Processing Abilities_Mathematical Reasoning
Test accuracy for NB is 0.6873385012919897
Test accuracy for SVC is 0.7596899224806202
Test accuracy for Logistic is 0.7416020671834626
Abilities_Number Facility [(0.0, 563), (0.74, 72), (1.0, 45), (0.5714285714285714, 33)]
2
... Processing Abilities_Number Facility
Test accuracy for NB is 0.6434108527131783
Test accuracy for SVC is 0.7441860465116279
Test accuracy for Logistic is 0.7080103359173127
Abilities_Oral Comprehension [(0.0, 428), (0.74, 93), (1.0, 56), (0.5714285714285714, 49)]
3
... Processing Abilities_Oral Comprehension
Test accuracy for NB is 0.49354005167958

## 3.3 Finally Creating the Y

In [ ]:
print(len(unlabeled_list))
print(len(labeled_list))


#Linear Support Vector Classification's Pipeline
SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words="english", ngram_range=(1, 2))),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])

Y_predicted = [unlabeled_list]
a=0
Y_skills_list = []
for skills in ONET_skills_column:
   
    if skills in Dict_SVC_accuracy.keys():
      test, train, X_test, X_train = test_train_df( pd.concat([ONET_df , LinkedIn_labeled_skills_df])   ,  LinkedIn_labeled_skills_df )   #Here test is not used, just passing for the function
      y = np.array([change_to_percentile(xi) for xi in train[skills]])
      SVC_pipeline.fit(X_train,  y  )
      prediction_SVC = SVC_pipeline.predict(unlabeled_list)
      Y_predicted.append(prediction_SVC)
      Y_skills_list.append(skills)


Y_values = []
for i in range(0,len(Y_predicted[0])):
    row = []
    for j in Y_predicted: row.append(j[i])
    Y_values.append(row)


Y_unlabeled = pd.DataFrame(Y_values, columns =  ["LinkedIn_Text"] + Y_skills_list  ) 
Y_labeled = LinkedIn_labeled_skills_df[["ONET_text"] + Y_skills_list]
Y_labeled = Y_labeled.rename(columns={"ONET_text": "LinkedIn_Text"}) 


# with pd.ExcelWriter(main_link + 'Input/Y_df.xlsx') as writer:  
#     Y.to_excel(writer, sheet_name='Sheet_name_1')


14182
1171


In [ ]:
Y  = pd.concat([Y_unlabeled  , Y_labeled]) 
with pd.ExcelWriter(main_link + 'Input/Y_df.xlsx') as writer:  
    Y.to_excel(writer, sheet_name='Sheet_name_1')



In [ ]:
Y_df = pd.read_csv(main_link + 'Input/Y_df.csv')[["Text"] + ONET_skills_column].rename(columns={"Text": "LinkedIn_Text"})
Y = csr_matrix(Y_df[ONET_skills_column].values)

How good is the classifier? Test using the training data

In [ ]:
print(len(ONET_skills_column))

In [ ]:
train, test = train_test_split(total_labeled_data, random_state=42, test_size=0.33, shuffle=True)
test = test[test["ONET_text"].isin(LinkedIn_labeled_skills_df['ONET_text'].values.tolist())]

X_train = train.ONET_text
X_test = test.ONET_text

print(len(X_test))
test_accuracy_NB ,test_accuracy_SVC ,test_accuracy_Logistic = [], [], []
a = 0

for skills in ONET_skills_column:
    print(a)
    a =a +1
    print('... Processing {}'.format(skills))

    # train the model using X_dtm & y
    y = np.array([change_to_percentile(xi) for xi in train[skills]])
    
    SVC_pipeline.fit(X_train,  y  )


    # compute the testing accuracy
    prediction_SVC = SVC_pipeline.predict(X_test)

    y = np.array([change_to_percentile(xi) for xi in test[skills]])

    SVC_accuracy = accuracy_score(y, prediction_SVC)
    print('Test accuracy for SVC is {}'.format(SVC_accuracy ))


    test_accuracy_SVC.append(SVC_accuracy)

    print("                 ")


print("SVC -> " + str(mean(test_accuracy_SVC)))



In [ ]:
arr = [1,4,5,7]
arr = np.asarray(test_accuracy_SVC)

min = np.amin(arr)
max = np.amax(arr)
range = np.ptp(arr)
variance = np.var(arr)
sd = np.std(arr)
 
print("Array =", arr)
print("Measures of Dispersion")
print("Minimum =", min)
print("Maximum =", max)
print("Range =", range)
print("Variance =", variance)
print("Standard Deviation =", sd)

Array = [1 4 5 7]
Measures of Dispersion
Minimum = 1
Maximum = 7
Range = 6
Variance = 4.6875
Standard Deviation = 2.165063509461097


# MLSI

# 4. Creating X

In [ ]:
import scipy
from scipy.sparse import csr_matrix

In [74]:
# tfidf_transformer=TfidfVectorizer(stop_words="english", ngram_range=(1,2)) 
# X = tfidf_transformer.fit_transform(unlabeled_list)
# print(type(X))

count_vector_transformer = CountVectorizer(stop_words="english", ngram_range=(1,2)) 
X = count_vector_transformer.fit_transform(unlabeled_list)
print(type(X))

X.get_shape()



<class 'scipy.sparse.csr.csr_matrix'>


(14182, 498067)

In [ ]:
# print(X[700])

In [78]:
a = dict(zip(count_vector_transformer.get_feature_names_out(), np.squeeze(np.asarray(X.sum(axis=0))).tolist() ))

In [85]:
count_vector_transformer.get_feature_names_out()

'40 active'

In [86]:
a["40 active"]

1

In [104]:
b = [x for x in count_vector_transformer.get_feature_names_out() if a[x] > 2]

In [105]:
print(len(b))
print(b[1000])

68758
4361 managerial


In [91]:
a["000 create"]

3

# 5. MLSI Function 

In [ ]:
from scipy.sparse.linalg import eigs
from scipy.sparse.linalg import inv
import time

In [ ]:
def get_M(K_x, C, gamma):

  t0 = time.time() # time
  C_inv = inv(C)

  t1 = time.time() # time
  print("Getting  C inverse "+str(t1- t0)) # time

  output = K_x * C_inv * K_x + gamma * K_x
  t2 = time.time() # time
  print("Getting  M "+str(t2- t1)) # time
  return output 

def GEV_solver(K_x,K_y,C, gamma, K):
  #Solve the generalized eigenvalue problem K_x^2 alpha = lambda [K_x C^{-1} K_x + gamma * K_x] alpha 

  t0 = time.time() # time
  K_x_sqr = K_x * K_x 
  t1 = time.time() # time
  print("Squaring "+str(t1- t0)) #time

  M_1 = get_M(K_x, C, gamma)

  t2 = time.time() # time
  vals, vecs = eigs(K_x_sqr , k=K, M = M_1)
  
  t3 = time.time() # time
  print("Solving eigenvalue problems "+str(t3- t2)) # time
  return vals, vecs

def get_MLSI_index(X, Y , beta, gamma, K):
  t0 = time.time() # time
  K_x = X * csr_matrix.transpose(X)
  K_y = Y * csr_matrix.transpose(Y)
  C = (1-beta) * K_x + beta * K_y 

  t1 = time.time() # time
  print("Step one "+str(t1- t0)) #time

  eigenvalues,  eigenvectors = GEV_solver(K_x,K_y,C, gamma, K)

  print(len(eigenvalues))
  print(len(eigenvectors))
  return eigenvalues,  eigenvectors 


random_sample = sample(range(0,14182),1000)
a,b = get_MLSI_index(X.tocsr()[random_sample ,:], Y.tocsr()[random_sample,:] , 0.5, 0, 6)

Step one 0.39394640922546387
Squaring 1.4939074516296387
Getting  C inverse 1.2770159244537354
Getting  M 3.0300114154815674
Solving eigenvalue problems 0.386918306350708
6
1000


In [ ]:
len(b[0])

6

In [ ]:

from random import sample
  

  
print(sample(range(0,100),3))

[44, 0, 45]


In [ ]:
# print(type(X))
# print(X.get_shape())
# X_T = csr_matrix.transpose(X)
# print(X_T.get_shape())
# print(type(X_T))

# beta = 0.5

# K_x = X * X_T
# print(type(K_x ))
# print(K_x.get_shape())

# # print(K_x)
# C = K_x * K_x
# print("----------------------------")
# # print(C)
# print(C.get_shape())
# print(type(C))

In [ ]:
# print(type(Y))
# print(Y.get_shape())
# Y_T = csr_matrix.transpose(Y)
# print(Y_T.get_shape())
# print(type(Y_T))

# YY_T = Y * Y_T
# print(YY_T.get_shape())

In [ ]:
# print(type(YY_T))

In [ ]:

  
# Create first csr matrix A
row_A = np.array([0, 0, 1, 2 ])
col_A = np.array([0, 1, 0, 1])
data_A = np.array([4, 3, 8, 9])
  
csrMatrix_A = csr_matrix((data_A, (row_A, col_A)),
                        shape = (3, 3))
  
# print first csr matrix
print("first csr matrix: \n", csrMatrix_A.toarray())
  
# Create second csr matrix B
row_B = np.array([0, 1, 1, 2 ])
col_B = np.array([0, 0, 1, 0])
data_B = np.array([7, 2, 5, 1])
  
csrMatrix_B = csr_matrix((data_B, (row_B, col_B)),
                        shape = (3, 3))
  
# print second scr matrix
print("second csr matrix:\n", csrMatrix_B.toarray())

print(type(csrMatrix_A )) 
print(type(csrMatrix_B )) 
# Multiply these matrices
sparseMatrix_AB = 2 * csrMatrix_A  + 3 * csrMatrix_B 
  
# print resultant matrix
print("Product Sparse Matrix:\n",sparseMatrix_AB.toarray() )

first csr matrix: 
 [[4 3 0]
 [8 0 0]
 [0 9 0]]
second csr matrix:
 [[7 0 0]
 [2 5 0]
 [1 0 0]]
<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
Product Sparse Matrix:
 [[29  6  0]
 [22 15  0]
 [ 3 18  0]]


In [ ]:
sparseMatrix_AB.tocsr()[sample(range(0,3),2),:].todense()
# sparseMatrix_AB.tocsr()[sample(range(0,3),1),:].get_shape()

matrix([[ 3, 18,  0],
        [22, 15,  0]])